# Install and import packages

In [3]:
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import optuna
from optuna.trial import TrialState
import os
import pandas as pd
import torch

from training_JUND import get_dataloader, Objective_LSTM, Objective_CNN, validate_epoch

/Users/danieljbpark/anaconda3/envs/csci4969/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
load_dotenv()

DATA_DIR_TRAIN = os.getenv("DATA_DIR_TRAIN")
DATA_DIR_VALIDATION = os.getenv("DATA_DIR_VALIDATION")
DATA_DIR_TEST = os.getenv("DATA_DIR_TEST")


# Train LSTM model and tune hyperparameters


In [ ]:

objective = Objective_LSTM(
    data_dir_train=DATA_DIR_TRAIN,
    data_dir_validation=DATA_DIR_VALIDATION,
)

study = optuna.create_study(direction="maximize")
study.optimize(func=objective, n_trials=20, timeout=1800, callbacks=[objective.callback])

# Evaluate best LSTM model's performance on test set

In [4]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

dataloader_test = get_dataloader(data_dir=DATA_DIR_TEST, batch_size=1024)
accuracy_test = validate_epoch(
    model=objective.best_model,
    data_loader=dataloader_test,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)
print("Accuracy of best model after hyperparameter tuning on test dataset: ", accuracy_test)

Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.7309251427650452
  Params: 
    epochs: 30
    LSTM_hidden_layer_size: 64
    MLP_hidden_layer_size: 32
    lr: 0.007721101990901195
Accuracy of best model after hyperparameter tuning on test dataset:  0.7308103442192078


# Train CNN model and tune hyperparameters

In [6]:
objective_cnn = Objective_CNN(
    data_dir_train=DATA_DIR_TRAIN,
    data_dir_validation=DATA_DIR_VALIDATION,
)

study_cnn = optuna.create_study(direction="maximize")
study_cnn.optimize(func=objective_cnn, n_trials=20, timeout=1800, callbacks=[objective_cnn.callback])

[I 2024-03-17 17:58:21,499] A new study created in memory with name: no-name-2a1500c7-1762-4e7b-8906-250360b615cd
[I 2024-03-17 18:02:12,431] Trial 0 finished with value: 0.7266857624053955 and parameters: {'epochs': 20, 'lr': 0.04040841784535208, 'conv_layer_1_num_channels': 4, 'conv_layer_1_kernel_size': 5, 'conv_layer_2_num_channels': 16, 'conv_layer_2_kernel_size': 9, 'MLP_hidden_layer_size': 64}. Best is trial 0 with value: 0.7266857624053955.
[I 2024-03-17 18:04:23,613] Trial 1 finished with value: 0.7245259284973145 and parameters: {'epochs': 10, 'lr': 0.010380926825107186, 'conv_layer_1_num_channels': 16, 'conv_layer_1_kernel_size': 3, 'conv_layer_2_num_channels': 16, 'conv_layer_2_kernel_size': 3, 'MLP_hidden_layer_size': 64}. Best is trial 0 with value: 0.7266857624053955.
[I 2024-03-17 18:09:15,376] Trial 2 finished with value: 0.7328217029571533 and parameters: {'epochs': 30, 'lr': 0.04886979430156351, 'conv_layer_1_num_channels': 8, 'conv_layer_1_kernel_size': 3, 'conv_lay

# Evaluate best CNN model's performance on test set

In [7]:
pruned_cnn_trials = study_cnn.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_cnn_trials = study_cnn.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study_cnn.trials))
print("  Number of pruned trials: ", len(pruned_cnn_trials))
print("  Number of complete trials: ", len(complete_cnn_trials))

print("Best trial:")
best_trial_cnn = study_cnn.best_trial

print("  Value: ", best_trial_cnn.value)

print("  Params: ")
for key, value in best_trial_cnn.params.items():
    print("    {}: {}".format(key, value))

dataloader_test = get_dataloader(data_dir=DATA_DIR_TEST, batch_size=1024)
cnn_accuracy_test = validate_epoch(
    model=objective_cnn.best_model,
    data_loader=dataloader_test,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)
print("Accuracy of best model after hyperparameter tuning on test dataset: ", cnn_accuracy_test)

Study statistics: 
  Number of finished trials:  11
  Number of pruned trials:  5
  Number of complete trials:  6
Best trial:
  Value:  0.7328217029571533
  Params: 
    epochs: 30
    lr: 0.04886979430156351
    conv_layer_1_num_channels: 8
    conv_layer_1_kernel_size: 3
    conv_layer_2_num_channels: 4
    conv_layer_2_kernel_size: 3
    MLP_hidden_layer_size: 32
Accuracy of best model after hyperparameter tuning on test dataset:  0.7373679876327515
